In [70]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [71]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [72]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

In [73]:
import matplotlib.pyplot as plt

nulos = pd.DataFrame(train_data.isna().sum() / train_data.isna().count(), columns=['Nulos'])
nulos.plot(kind='bar', figsize=(14,4))

In [74]:
train_data[train_data['Embarked'].isna()].head()

In [75]:
df_fares = pd.DataFrame(train_data['Fare'].unique(), columns=['Fare'])
df_fares.hist(figsize=(14,4),column='Fare')

In [76]:
train_data[train_data['Fare']>200].head()

In [77]:
train_data['Embarked'].unique()

In [78]:
train_data['SibSp'].unique()

In [79]:
train_data['Parch'].unique()

In [80]:
train_data_clean = train_data[['Survived','Pclass','Sex','Age','SibSp','Parch','Embarked']]
train_data_clean = train_data_clean[train_data['Age'].notna()]
train_data_clean = train_data_clean[train_data['Embarked'].notna()]

In [81]:
import seaborn as sns
from sklearn import preprocessing

column_Sex = preprocessing.LabelEncoder()
column_Sex.fit(train_data_clean['Sex'])
train_data_clean['Sex_converted'] = column_Sex.transform(train_data_clean['Sex'])

column_Embarked = preprocessing.LabelEncoder()
column_Embarked.fit(train_data_clean['Embarked'])
train_data_clean['Embarked_converted'] = column_Embarked.transform(train_data_clean['Embarked'])

train_data_converted = train_data_clean[['Survived','Pclass','Sex_converted','SibSp','Parch','Embarked_converted']]

plt.figure(figsize=(5,5))
cor = train_data_converted.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [82]:
column_Sex_test = preprocessing.LabelEncoder()
column_Sex_test.fit(test_data['Sex'])
test_data['Sex_converted'] = column_Sex_test.transform(test_data['Sex'])

column_Embarked_test = preprocessing.LabelEncoder()
column_Embarked_test.fit(test_data['Embarked'])
test_data['Embarked_converted'] = column_Embarked_test.transform(test_data['Embarked'])

In [121]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

feature_Survived = 'Survived'
features = ['Pclass','Sex_converted','Embarked_converted']
features_selected = [feature_Survived,'Pclass','Sex_converted','Embarked_converted']
train_data_selected = train_data_converted[features_selected]

Y_train = train_data_selected[feature_Survived]
#Y_test = test_data[feature_Survived]

test_data_selected = test_data[features]
X_train = pd.get_dummies(train_data_selected[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X_train, Y_train)
X_pred = model.predict(X_test)

Y_pred = model.predict(X_train)
print(classification_report(Y_train, Y_pred))

In [122]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': X_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")